In [22]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUDProject2Final import AnimalShelter

###########################
# Data Manipulation / Model
###########################

shelter = AnimalShelter("aacuser", "SNHUPASS", "localhost", 27017, "AAC", "animals")

# Connect to database via CRUD Module
db = shelter

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#   html.Div(id='hidden-div', style={'display':'none'}),
    html.A([html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),height = 250, width = 250))]),
    html.Hr(),
    html.H2('Thomas Comer: 7-2 CS-340 \n Username: tcomer862631780'),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label':'Water Rescue', 'value': 'Water'},
            {'label':'Mountain or Wilderness Rescue', 'value': 'Mountain'},
            {'label':'Disaster Rescue or Individual Tracking', 'value':'Disaster'},
        ],
        value='All'
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=True,
        row_selectable="single", #allows user to select a row (required to allow mapping functionality
        selected_rows=[0], #initilizes selected row as first row
        filter_action="native", #creates a filter for data
        sort_action="native", #creates a sorting feature
        page_action="native", #page selection
        page_current=0, #always starts the program on the first page
        page_size=10, #limites to 10 entries per page
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    #set up an if/else if/else block to respond to the radio buttons
    if filter_type == 'All':
        df = pd.DataFrame.from_records(shelter.read({}))
    elif filter_type == 'Water':

        df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Female'},
          {'$or': [
              {'breed': 'Labrador Retriever Mix'},
              {'breed': 'Chesa Bay Retr Mix'},
              {'breed': 'Newfoundland Mix'},
              {'breed': 'Newfoundland/Labrador Retriever'},
              {'breed': 'Newfoundland/Australian Cattle Dog'},
              {'breed': 'Newfoundland/Great Pyrenees'}]
          },
          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
              {'age_upon_outcome_in_weeks': {'$lte': 156}}]
          }]
        })))
    elif filter_type == 'Mountain':
        
        df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
          {'$or': [
              {'breed': 'German Shepherd'},
              {'breed': 'Alaskan Malamute'},
              {'breed': 'Old English Sheepdog'},
              {'breed': 'Rottweiler'},
              {'breed': 'Siberian Husky'}]
          },
          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
              {'age_upon_outcome_in_weeks': {'$lte': 156}}]
          }]
        })))
    elif filter_type == 'Disaster':
        df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
          {'$or': [
              {'breed': 'Doberman Pinscher'},
              {'breed': 'German Shepherd'},
              {'breed': 'Golden Retriever'},
              {'breed': 'Bloodhound'},
              {'breed': 'Rottweiler'}]
          },
          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
              {'age_upon_outcome_in_weeks': {'$lte': 300}}]
          }]
        })))
    else:
        raise Exception("Unknown filter")
    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')        
        
    return (data,columns)

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #

    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]


# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server() 

Dash app running on https://forestsecure-tangotonight-3000.codio.io/proxy/8050/
